# Part 1. #
## This solution makes use of code for wiki table to csv scraping developed by Andy Roche https://github.com/rocheio/wiki-table-scrape ##

In [2]:
#Code for scraping tables to csv

import csv
import os

from bs4 import BeautifulSoup
import requests


def scrape(url, output_name):
    """Create CSVs from all tables in a Wikipedia article.
    ARGS:
        url (str): The full URL of the Wikipedia article to scrape tables from.
        output_name (str): The base file name (without filepath) to write to.
    """

    # Read tables from Wikipedia article into list of HTML strings
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "lxml")
    table_classes = {"class": ["sortable", "plainrowheaders"]}
    wikitables = soup.findAll("table", table_classes)

    # Create folder for output if it doesn't exist
    os.makedirs(output_name, exist_ok=True)

    for index, table in enumerate(wikitables):
        # Make a unique file name for each CSV
        if index == 0:
            filename = output_name
        else:
            filename = output_name + "_" + str(index)

        filepath = os.path.join(output_name, filename) + ".csv"

        with open(filepath, mode="w", newline="", encoding="utf-8") as output:
            csv_writer = csv.writer(output, quoting=csv.QUOTE_ALL, lineterminator="\n")
            write_html_table_to_csv(table, csv_writer)


def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.
    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """

    # Hold elements that span multiple rows in a list of
    # dictionaries that track 'rows_left' and 'value'
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])

        # If the first row, use it to define width of table
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        # Insert values from cells that span into this row
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    # Insert the data from previous row; decrement rows left
                    value = rowspan_data["value"]
                    cells.insert(index, value)

                    if saved_rowspans[index]["rows_left"] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]["rows_left"] -= 1

        # If an element with rowspan, save it for future cells
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {"rows_left": int(cell["rowspan"]), "value": cell}
                saved_rowspans[index] = rowspan_data

        if cells:
            # Clean the data of references and unusual whitespace
            cleaned = clean_data(cells)

            # Fill the row with empty columns if some are missing
            # (Some HTML tables leave final empty cells without a <td> tag)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing

            writer.writerow(cleaned)


def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.
    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.
    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """

    cleaned_cells = []

    for cell in row:
        # Strip references from the cell
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()

        # Strip sortkeys from the cell
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()

        # Strip footnotes from text and join into a single string
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != "["]

        cleaned = (
            "".join(no_footnotes)  # Combine elements into single string
            .replace("\xa0", " ")  # Replace non-breaking spaces
            .replace("\n", " ")  # Replace newlines
            .strip()
        )

        cleaned_cells += [cleaned]

    return cleaned_cells

In [3]:
#scraping relevant table
scrape(
    url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",
    output_name="postal"
)

In [4]:
import pandas as pd
import numpy as np

#reading scraped table from csv file
df=pd.read_csv('postal/postal.csv')
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df2 = df[df.Borough != 'Not assigned'].reset_index(drop=True) #dropping rows with postcodes unassigned to any borough

for n in range(df2.shape[0]):
    if df2.iloc[n,2]=="Not assigned":
        df2.iloc[n,2]=df2.iloc[n,1]    #Not assigned neighbourhood get same names as boroughs
    df2.iloc[n,2]+=', '   #adding comma and space for grouping purpouse later

df2.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M5A,Downtown Toronto,"Regent Park,"
4,M6A,North York,"Lawrence Heights,"
5,M6A,North York,"Lawrence Manor,"
6,M7A,Queen's Park,"Queen's Park,"
7,M9A,Etobicoke,"Islington Avenue,"
8,M1B,Scarborough,"Rouge,"
9,M1B,Scarborough,"Malvern,"


In [6]:
agg_fcns={'Borough':'first', 'Neighbourhood':sum}
df3=df2.groupby(df2['Postcode']).aggregate(agg_fcns).reset_index()   #combining neighborhoods with same postal codes

for n in range(df3.shape[0]):
    df3.iloc[n,2]=df3.iloc[n,2][0:-2]   #removing comma and space at the end of neighborhood names

df3.head(10)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
df3.shape

(103, 3)

# Part 2: Assigning coordinates to postal codes #

In [23]:
latln=pd.read_csv('https://cocl.us/Geospatial_data') #importing coordinates from csv file

In [24]:
latln.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
